# Imports and Prep

## imports

In [ ]:
import json
import requests

import pandas as pd

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import datetime
import dateutil


## functions and variables needed

In [ ]:
# unauthenticated requests have lower rate limit - this is why we authenticate
github_user_name = "UPDATE_WITH_YOUR_USERNAME"
#TODO: set to your GitHub username
github_oauth_token = "UPDATE_WITH_YOUR_TOKEN"
#TODO: set to your personal GitHib OAuth token  # get one here: https://github.com/settings/tokens

min_fetch_created_datetime = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=-180), "%Y-%m-%d") + "T00:00:00Z"
print(min_fetch_created_datetime)

min_report_created_datetime = datetime.datetime.now() + datetime.timedelta(days=-90)
min_report_created_datetime_str = datetime.datetime.strftime(min_report_created_datetime, "%Y-%m-%d") + "T00:00:00Z"
print(min_report_created_datetime_str)


In [ ]:
# issue functions
def send_request_to_github_api(url):
    # We use basic auth with a personal OAuth token: https://developer.github.com/v3/auth/#basic-authentication
    
    headers = {'Accept': 'application/vnd.github.v3+json'
               }
    
    response = requests.get(url,
                            auth=(github_user_name, github_oauth_token),
                            headers=headers)
    
    if response.status_code != 200:
        raise Exception("HTTP Status: {0:s}, Reason: {1:s}".format(str(response.status_code), response.reason))
    
    return response

def sunday_of_the_week_date(dt):
#     sunday_dt = (dt + datetime.timedelta(
#                 days=(6 - dt.weekday()), weeks=-1)).replace(hour=0, minute=0, second=0, microsecond=0)
    if not dt:
        return None
    elif dt.weekday() == 6:
        sunday_dt = dt.replace(hour=0, minute=0, second=0, microsecond=0)
    else:
        sunday_dt = (dt + datetime.timedelta(
                days=(6 - dt.weekday()), weeks=-1)).replace(hour=0, minute=0, second=0, microsecond=0)

    return sunday_dt

def sunday_of_the_week_date_str(dt):
    if not dt:
        return None
    elif dt.weekday() == 6:
        sunday_dt = dt.replace(hour=0, minute=0, second=0, microsecond=0)
    else:
        sunday_dt = (dt + datetime.timedelta(
                days=(6 - dt.weekday()), weeks=-1)).replace(hour=0, minute=0, second=0, microsecond=0)
    return datetime.datetime.strftime(sunday_dt, "%Y-%m-%d")

def get_all_info_for_issue_row(issue_info):
    issue_close_datetime = get_issue_close_datetime(issue_info)
    first_core_team_issue_comment_datetime = get_first_core_team_issue_comment_datetime(issue_info)
    
    return {
        "number": issue_info["number"],
        "author_login": issue_info["user"]["login"],
        "created_dt": issue_info["created_at"],
        "is_core_engineering_queue": has_label_with_name(issue_info, "core-engineering-queue"),
        "is_help_wanted": has_label_with_name(issue_info, "help_wanted"),
        "is_devrel": has_label_with_name(issue_info, "devrel"),
        "is_triage": has_label_with_name(issue_info, "triage"),
        "first_core_team_comment_dt": first_core_team_issue_comment_datetime,
        "close_dt": issue_close_datetime,
        "priority_category": get_issue_category_from_labels(issue_info)
    }
    
def get_issue_close_datetime(issue_info):
    return issue_info["closed_at"]

def has_label_with_name(issue_info, label_name):
    issue_labels = issue_info.get("labels")
    if issue_labels is not None:
        return len([issue_label for issue_label in issue_labels if issue_label["name"] == label_name]) > 0
    return False
    

def get_issue_category_from_labels(issue_info):
    issue_labels = issue_info.get("labels")
    if issue_labels is not None:
        for issue_label in issue_labels:
            if issue_label["name"] == 'core-engineering-queue':
                return 'core-engineering-queue'
            if issue_label["name"] == 'help wanted':
                return 'help wanted'
            if issue_label["name"] == 'devrel':
                return 'devrel'
            if issue_label["name"] == 'triage':
                return 'triage'
            
    return "other"


def get_first_core_team_issue_comment_datetime(issue_info):
    issue_number = issue_info["number"]
    first_core_team_issue_comment_datetime = None
    
    issue_comments_response = send_request_to_github_api("https://api.github.com/repos/great-expectations/great_expectations/issues/{0:d}/comments".format(issue_number))

    core_team_issue_comment_datetimes = sorted([comment["created_at"] for comment in issue_comments_response.json() if comment["user"]["login"] in core_team_usernames])
    
    if len(core_team_issue_comment_datetimes) > 0:
        first_core_team_issue_comment_datetime = core_team_issue_comment_datetimes[0]

    return first_core_team_issue_comment_datetime
    

In [ ]:
# PR functions
def get_all_info_for_pr_row(pr_info):
    pr_merge_datetime = get_pr_merge_datetime(pr_info)
    pr_close_datetime = get_pr_close_datetime(pr_info)
    first_core_team_pr_comment_datetime = get_first_core_team_pr_comment_datetime(pr_info)
    first_core_team_pr_review_datetime, first_core_team_approval_datetime = get_first_core_team_pr_review_and_approval_datetimes(pr_info)

    if first_core_team_pr_comment_datetime is not None and first_core_team_pr_review_datetime is not None:
        first_core_team_reaction_datetime = sorted([first_core_team_pr_comment_datetime, first_core_team_pr_review_datetime])[0]
    else: 
        first_core_team_reaction_datetime = first_core_team_pr_comment_datetime or first_core_team_pr_review_datetime
    
    return {
        "number": pr_info["number"],
        "created_dt": pr_info["created_at"],
        "first_core_team_reaction_dt": first_core_team_reaction_datetime,
        "first_core_team_comment_dt": first_core_team_pr_comment_datetime,
        "first_core_team_review_dt": first_core_team_pr_review_datetime,
        "first_core_team_approval_dt": first_core_team_approval_datetime,
        "merge_dt": pr_merge_datetime,
        "close_dt": pr_close_datetime,
    }
    
    
def get_pr_close_datetime(pr_info):
    return pr_info["closed_at"]

def get_pr_merge_datetime(pr_info):
    return pr_info["merged_at"]
    
def get_first_core_team_pr_comment_datetime(pr_info):
    pr_number = pr_info["number"]
    first_core_team_pr_comment_datetime = None
    
    pr_comments_response = send_request_to_github_api("https://api.github.com/repos/great-expectations/great_expectations/issues/{0:d}/comments".format(pr_number))

    core_team_pr_comment_datetimes = sorted([comment["created_at"] for comment in pr_comments_response.json() if comment["user"]["login"] in core_team_usernames])
    
    if len(core_team_pr_comment_datetimes) > 0:
        first_core_team_pr_comment_datetime = core_team_pr_comment_datetimes[0]

    return first_core_team_pr_comment_datetime
    
def get_first_core_team_pr_review_and_approval_datetimes(pr_info):
    pr_number = pr_info["number"]
    first_core_team_pr_review_datetime = None
    first_core_team_pr_approval_datetime = None
    
    pr_reviews_response = send_request_to_github_api("https://api.github.com/repos/great-expectations/great_expectations/pulls/{0:d}/reviews".format(pr_number))
    
    core_team_pr_review_datetimes = sorted([review["submitted_at"] for review in pr_reviews_response.json() if review["user"]["login"] in core_team_usernames])
    if len(core_team_pr_review_datetimes) > 0:
        first_core_team_pr_review_datetime = core_team_pr_review_datetimes[0]

    core_team_pr_approval_datetimes = sorted([review["submitted_at"] for review in pr_reviews_response.json() if review["user"]["login"] in core_team_usernames and review["state"] == "APPROVED"])
    if len(core_team_pr_approval_datetimes) > 0:
        first_core_team_pr_approval_datetime = core_team_pr_approval_datetimes[0]

    return (first_core_team_pr_review_datetime, first_core_team_pr_approval_datetime)
    

    

## core team list

In [ ]:
core_team_usernames = { # these are former team members to filter out. We will additionally add current members of the organization
    'spbail-ge',
    'spbail',
    'Aylr',
    'dependabot[bot]',
    'gilpasternak35',
    'snyk-bot',
    'dependabot'
}

url_sc = "https://api.github.com/orgs/superconductive/members"
url_ge = "https://api.github.com/orgs/great-expectations/members"

for member_organization_url in [url_sc, url_ge]:
    members = send_request_to_github_api(member_organization_url)
    core_team_usernames.update({i.get("login") for i in members.json()})

core_team_usernames

# Contributor Stats

In [ ]:
response = requests.get("https://api.github.com/repos/great-expectations/great_expectations/stats/contributors")
response_json = response.json()
response_json

In [ ]:
contributor_list = []

for contrib in response_json:
    contributor_list.append(contrib["author"])

contributor_list

In [ ]:
contributor_df = pd.DataFrame(contributor_list)
contributor_df

In [ ]:
contributor_week_list = []
all_weeks = set()

for contrib in response_json:
    
    first_contrib = True
    for week in contrib["weeks"]:
        week["login"] = contrib["author"]["login"]
        week["contributors"] = 1
    
        if week["c"] > 0:
            week["first_contrib"] = first_contrib
            contributor_week_list.append(week)
            
            #This method of detecting first contributions will only work if week timestamps in "w" are sorted in ascending order
            #This appears to be true, but is untested.
            first_contrib = False
            
        all_weeks.add(week["w"])
        
contributor_week_df = pd.DataFrame(contributor_week_list)
contributor_week_df

In [ ]:
filtered_contributor_week_df = contributor_week_df[contributor_week_df.login.isin(core_team_usernames)==False]
filtered_contributor_week_df

In [ ]:
week_df = pd.DataFrame({"week_timestamp" : list(all_weeks)})

contributions = filtered_contributor_week_df.groupby("w").sum()["c"].to_frame().reset_index()
contributions.columns = ["week_timestamp", "total_contributions"]
week_df = week_df.merge(contributions, how='left', on="week_timestamp")

contributors = filtered_contributor_week_df.groupby("w").sum()["contributors"].to_frame().reset_index()
contributors.columns = ["week_timestamp", "total_contributors"]
week_df = week_df.merge(contributors, how='left', on="week_timestamp")

contributors = filtered_contributor_week_df.groupby("w").sum()["first_contrib"].to_frame().reset_index()
contributors.columns = ["week_timestamp", "new_contributors"]
week_df = week_df.merge(contributors, how='left', on="week_timestamp")

week_df = week_df.fillna(0)

week_df["week_dates"] = week_df.week_timestamp.map(lambda x: datetime.datetime.fromtimestamp(x))

week_df.head()

In [ ]:
week_df.sort_values('week_timestamp', inplace=True)
week_df.week_timestamp[:13]

# plt.plot(week_df.week_timestamp[-13:], week_df.total_contributions[-13:])
# plt.scatter(week_df.week_timestamp[-13:], week_df.total_contributions[-13:])
# plt.xticks(week_df.week_timestamp[-13:], week_df.week_dates[-13:].map(lambda x: str(x)[:11]), rotation=45, ha="right")
# plt.title("Outside contributions (commits) per week")

# plt.savefig("figures/commits_per_week_1q_trailing_"+str(datetime.date.today())+".png")

In [ ]:
plt.bar(range(13), week_df.total_contributors[-13:], width=0.45)
plt.xticks(range(13), week_df.week_dates[-13:], rotation='vertical')

plt.bar(range(13), week_df.new_contributors[-13:], width=0.45)
plt.xticks(range(13), week_df.week_dates[-13:], rotation='vertical')
plt.title('Contributors per week')

plt.legend(["Unique", "New"])

# plt.savefig("figures/contributors_per_week_1q_trailing_"+str(datetime.date.today())+".png")

# PR Stats

In [ ]:
list_pr_request_url = "https://api.github.com/repos/great-expectations/great_expectations/pulls?state=all&sort=created&direction=desc"
pr_infos = []
# we are using GH API's paging
while list_pr_request_url is not None :
    pr_list_response = send_request_to_github_api(list_pr_request_url)

    pr_infos.extend(pr_list_response.json())

    # response.links has "next" {'url': '...&page=3', 'rel': 'next'}
    next_page_info = pr_list_response.links.get("next")
    if next_page_info is None:
        break
    list_pr_request_url = next_page_info["url"]

    sorted_page_pr_create_datetimes = sorted([pr_info["created_at"] for pr_info in pr_list_response.json()])
    print("Paged response: {0:d} PRs, min created at: {1:s}".format(len(sorted_page_pr_create_datetimes), sorted_page_pr_create_datetimes[0] if len(sorted_page_pr_create_datetimes) > 0 else "")) 
    if len(sorted_page_pr_create_datetimes) > 0 and sorted_page_pr_create_datetimes[0] < min_report_created_datetime_str:
        break

#eligible_pr_infos = [pr_info for pr_info in pr_infos if (pr_info["user"]["login"] not in core_team_usernames and not pr_info["draft"] and pr_info["created_at"] >= min_pr_created_datetime )]
eligible_pr_infos = [pr_info for pr_info in pr_infos if (pr_info["user"]["login"] not in core_team_usernames and not pr_info["draft"] and pr_info["created_at"] >= str(min_report_created_datetime) )]
print("Fetched {0:d} PRs. Out of them {1:d} eligible for the report.".format(len(pr_infos), len(eligible_pr_infos)))
        

In [ ]:
pr_info_rows = [get_all_info_for_pr_row(pr_info) for pr_info in eligible_pr_infos]

df_prs = pd.DataFrame(pr_info_rows)
df_prs.head()

In [ ]:
df_prs["week"] = df_prs.apply(lambda x: sunday_of_the_week_date_str(dateutil.parser.parse(x["created_dt"])), axis=1) 
df_prs["week_dt"] = df_prs.apply(lambda x: sunday_of_the_week_date(dateutil.parser.parse(x["created_dt"])), axis=1) 
df_prs = df_prs[(df_prs["close_dt"].isnull()) | (df_prs["merge_dt"].notnull())]

df_prs["created"] = df_prs["created_dt"].notnull().astype(int)
df_prs["first_core_team_reaction"] = df_prs["first_core_team_reaction_dt"].notnull().astype(int)
df_prs["first_core_team_review"] = df_prs["first_core_team_review_dt"].notnull().astype(int)
df_prs["first_core_team_approval"] = df_prs["first_core_team_approval_dt"].notnull().astype(int)
df_prs["merge"] = df_prs["merge_dt"].notnull().astype(int)
df_prs["close"] = df_prs["close_dt"].notnull().astype(int)
df_prs

df_report = df_prs.pivot_table(index=["week"], values=["created", "first_core_team_reaction", "first_core_team_review", "first_core_team_approval", "merge"], aggfunc=[np.sum])
df_report = df_report.reset_index()
df_report.columns = ["week", "created", "approved", "reacted", "reviewed", "merged"]
df_report

In [ ]:
from matplotlib.lines import Line2D
plt.rcParams["figure.figsize"] = (20,7)
_, ax = plt.subplots()
# df_report[df_report["week"]>"2020-09-27"].plot(kind= 'bar', x="week", y=["created", "reacted", "reviewed", "approved", "merged"])
#df_report.plot(kind= 'bar', x="week", y=["created", "reacted", "reviewed", "approved", "merged"])
plt.legend(bbox_to_anchor=(1.0, 1), loc='upper left')
df_report.created.plot(kind='bar', ax=ax, color='red')
df_report.reacted.plot(kind='bar', ax=ax, color='grey')
df_report.reviewed.plot(kind='bar', ax=ax, color='blue')
df_report.approved.plot(kind='bar', ax=ax, color='green')
df_report.merged.plot(kind='bar', ax=ax, color='purple')
ax.set_xticks(df_report.index)
ax.set_xticklabels(df_report["week"].values)
cmap = plt.cm.coolwarm
custom_lines = [Line2D([0], [0], color="red", lw=10),
                Line2D([0], [0], color="grey", lw=10),
                Line2D([0], [0], color="blue", lw=10),
                Line2D([0], [0], color="green", lw=10),
                Line2D([0], [0], color="purple", lw=10),
               ]
ax.legend(custom_lines, ['No action taken', 'Acknlowledged', 'Reviewed', 'Approved', 'Merged'])
plt.title("Community PRs - last 3 months, by weekly cohort", fontsize=25)


In [ ]:
def pr_is_open_during_week(week_dt, x):
    week_end_dt = week_dt+datetime.timedelta(days = 7)
    created_dt = dateutil.parser.parse(x["created_dt"])
    close_dt = None
    if not pd.isnull(x["close_dt"]):
        close_dt = dateutil.parser.parse(x["close_dt"])
    
    return created_dt < week_end_dt and (pd.isnull(close_dt) or close_dt > week_end_dt)
    

# This is a not a Pandas way to do it, but gets the job done. Would be lovely to find out an elegant way to do this.    
week_open_pr_counts = []    
for week_dt_ref in df_prs["week_dt"].unique():
    s_tmp = df_prs.apply(lambda x: pr_is_open_during_week(week_dt_ref, x), axis=1)
    week_open_pr_counts.append({
        "week_dt": week_dt_ref,
        "count": s_tmp[s_tmp == True].shape[0]
    })

df_num_open_community_prs_weekly = pd.DataFrame(week_open_pr_counts).sort_values("week_dt")
df_num_open_community_prs_weekly["week"] = df_num_open_community_prs_weekly["week_dt"].apply(lambda x: datetime.datetime.strftime(x, "%Y-%m-%d"))
pd.DataFrame(week_open_pr_counts)

In [ ]:
from matplotlib.lines import Line2D
plt.rcParams["figure.figsize"] = (20,7)
_, ax = plt.subplots()
plt.legend(bbox_to_anchor=(1.0, 1), loc='upper left')
df_num_open_community_prs_weekly.plot(kind='line', ax=ax, x="week", y="count", rot=90)
ax.set_xticks(df_report.index)
ax.set_xticklabels(df_num_open_community_prs_weekly["week"].values)
cmap = plt.cm.coolwarm
plt.title('Number of open community PRs', fontsize=25)


In [ ]:
# identify the week a PR was created, merged, and closed if not merged
pr_burn_date_df = pd.DataFrame(pr_info_rows)
pr_burn_date_df = pr_burn_date_df[["created_dt", "merge_dt", "close_dt"]]
pr_burn_date_df["created_this_week"] = pr_burn_date_df.apply(lambda x: sunday_of_the_week_date_str(dateutil.parser.parse(x["created_dt"])), axis=1)
pr_burn_date_df["merged_this_week"] = pr_burn_date_df.apply(lambda x: sunday_of_the_week_date_str(dateutil.parser.parse(x["merge_dt"]) if x["merge_dt"] is not None else None), axis=1)
pr_burn_date_df["closed_this_week"] = pr_burn_date_df.apply(lambda x: sunday_of_the_week_date_str(dateutil.parser.parse(x["close_dt"]) if x["close_dt"] is not None else None), axis=1)
pr_burn_date_df = pr_burn_date_df[["created_this_week", "merged_this_week", "closed_this_week"]]
pr_burn_date_df["closed_no_merge_this_week"] = np.where(pr_burn_date_df["merged_this_week"].isnull() & pr_burn_date_df["closed_this_week"].notnull(), pr_burn_date_df["closed_this_week"], None)

# count each category out
all_date_values = pr_burn_date_df.values.ravel()
unique_dates = pd.unique(all_date_values)
unique_dates = np.sort(unique_dates[unique_dates != np.array(None)])
pr_burn_df = pd.DataFrame(data=unique_dates, columns=["week"])
created_count = pr_burn_date_df["created_this_week"].value_counts()
merged_count = pr_burn_date_df["merged_this_week"].value_counts()
closed_no_merge_count = pr_burn_date_df["closed_no_merge_this_week"].value_counts()
pr_burn_df = pr_burn_df.merge(created_count, how="outer", left_on="week", right_index=True)
pr_burn_df = pr_burn_df.merge(merged_count, how="outer", left_on="week", right_index=True)
pr_burn_df = pr_burn_df.merge(closed_no_merge_count, how="outer", left_on="week", right_index=True)
pr_burn_df = pr_burn_df.fillna(0)
pr_burn_df["merged_or_closed_this_week"] = pr_burn_df["merged_this_week"] + pr_burn_df["closed_no_merge_this_week"]

# calculate cumulative open PRs
open_prs_start_of_first_week = 21
pr_burn_df["change_in_open_prs"] = pr_burn_df["created_this_week"] - pr_burn_df["merged_or_closed_this_week"]
pr_burn_df["cumulative_open_prs"] = pr_burn_df["change_in_open_prs"].cumsum()
pr_burn_df["cumulative_open_prs"] = pr_burn_df["cumulative_open_prs"] + open_prs_start_of_first_week
pr_burn_df = pr_burn_df.iloc[-13:]
pr_burn_df

In [ ]:
# dataframe just for plot
pr_plot_df = pr_burn_df[["week", "created_this_week", "merged_or_closed_this_week", "change_in_open_prs", "cumulative_open_prs"]]
pr_plot_df["merged_or_closed_this_week"] = pr_plot_df["merged_or_closed_this_week"] * -1
column_names = ["Week", "Created this Week", "Merged or Closed this Week", "Change in Open PRs", "Cumulative Open PRs"]
pr_plot_df.columns = column_names

# plot
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={"height_ratios": [4, 1]})

pr_plot_df.plot(x="Week", y="Cumulative Open PRs", kind="line", ax=ax1, drawstyle='steps-mid', color="black")
pr_plot_df.plot(x="Week", y="Created this Week", kind="bar", ax=ax1, color="salmon", bottom=pr_plot_df["Cumulative Open PRs"])
pr_plot_df.plot(x="Week", y="Merged or Closed this Week", kind="bar", ax=ax1, color="lightgreen", bottom=pr_plot_df["Cumulative Open PRs"])
pr_plot_df["min"] = pr_plot_df["Cumulative Open PRs"] + pr_plot_df["Merged or Closed this Week"]
pr_plot_df["max"] = pr_plot_df["Cumulative Open PRs"] + pr_plot_df["Created this Week"]
ax1.set_ylim([pr_plot_df["min"].min()-3, pr_plot_df["max"].max()+6])
pr_plot_df = pr_plot_df.drop(axis=1, columns=["max", "min"])
ax1.get_xaxis().set_visible(False)

change_colors = np.where(pr_plot_df["Change in Open PRs"] <= 0, "lightgreen", "salmon")
markers = np.where(pr_plot_df["Change in Open PRs"] <= 0, "v", "^")

pr_plot_df["dummy"] = [5] * len(pr_plot_df)
for i in range(len(pr_plot_df)):
    ax2.set_ylim([-5,10])
    ax2.scatter(x=pr_plot_df["Week"].iloc[i], y=pr_plot_df["dummy"].iloc[i], marker=markers[i], color=change_colors[i], s=500, edgecolors="black")
    ax2.annotate(text=pr_plot_df["Change in Open PRs"].iloc[i].astype(int), xy=(pr_plot_df["Week"].iloc[i], -2), ha="center", fontsize=20)
pr_plot_df = pr_plot_df.drop(axis=1, columns=["dummy"])
ax2.get_yaxis().set_visible(False)

fig.tight_layout()
ax1.grid(b=True, axis="y")
ax1.tick_params(axis="y", which="major", labelsize=16)
ax1.legend(loc=2, fontsize=16)
ax1.set_title('Cumulative Change in Open Community PRs (Previous 13 Weeks)', fontdict={'fontsize': 25})
ax2.tick_params(axis="x", which="major", labelsize=16, rotation=60)
plt.subplots_adjust(hspace=0)
plt.show()

# Issue Stats

In [ ]:
list_issues_request_url = "https://api.github.com/repos/great-expectations/great_expectations/issues?state=all&sort=created&direction=desc"
issue_infos = []
# we are using GH API's paging
while list_issues_request_url is not None :
    issue_list_response = send_request_to_github_api(list_issues_request_url)

    issue_infos.extend(issue_list_response.json())

    # response.links has "next" {'url': '...&page=3', 'rel': 'next'}
    next_page_info = issue_list_response.links.get("next")
    if next_page_info is None:
        break
    list_issues_request_url = next_page_info["url"]

    sorted_page_pr_create_datetimes = sorted([pr_info["created_at"] for pr_info in issue_list_response.json()])
    print("Paged response: {0:d} issues, min created at: {1:s}".format(len(sorted_page_pr_create_datetimes), sorted_page_pr_create_datetimes[0] if len(sorted_page_pr_create_datetimes) > 0 else "")) 
    if len(sorted_page_pr_create_datetimes) > 0 and sorted_page_pr_create_datetimes[0] < min_fetch_created_datetime:
        break

eligible_issue_infos = [issue_info for issue_info in issue_infos if (not issue_info.get("pull_request") and issue_info["created_at"] >= min_fetch_created_datetime )]
print("Fetched {0:d} issues. Out of them {1:d} eligible for the report.".format(len(issue_infos), len(eligible_issue_infos)))
        

In [ ]:
issue_info_rows = [get_all_info_for_issue_row(issue_info) for issue_info in eligible_issue_infos]
df_issues = pd.DataFrame(issue_info_rows)
df_issues.head()

In [ ]:
df_issues["week"] = df_issues.apply(lambda x: sunday_of_the_week_date_str(dateutil.parser.parse(x["created_dt"])), axis=1) 
df_issues["week_dt"] = df_issues.apply(lambda x: sunday_of_the_week_date(dateutil.parser.parse(x["created_dt"])), axis=1) 
df_issues.head()

In [ ]:
df_issues["created"] = df_issues["created_dt"].notnull().astype(int)
df_issues["first_core_team_comment"] = (df_issues["first_core_team_comment_dt"].notnull() | df_issues["close_dt"].notnull()).astype(int)
df_issues["close"] = df_issues["close_dt"].notnull().astype(int)

df_community_reported_issues = df_issues[df_issues.apply(lambda x: x["author_login"] not in core_team_usernames, axis=1)] 
print("all issues", df_issues.shape)
print("community issues", df_community_reported_issues.shape)

In [ ]:
df_report = df_community_reported_issues[df_community_reported_issues["created_dt"] > min_report_created_datetime_str].pivot_table(index=["week"], values=["created", "first_core_team_comment", "close"], aggfunc=[np.sum])
df_report = df_report.reset_index()
df_report.columns = ["week", "closed", "created", "responded"]
df_report.head()

In [ ]:
from matplotlib.lines import Line2D
#plt.rcParams.update({'axes.titlesize': 'small'})
plt.rcParams["figure.figsize"] = (20,7)
_, ax = plt.subplots()
plt.legend(bbox_to_anchor=(1.0, 1), loc='upper left')
df_report.created.plot(kind='bar', ax=ax, color='red')
df_report.responded.plot(kind='bar', ax=ax, color='grey')
df_report.closed.plot(kind='bar', ax=ax, color='purple')
ax.set_xticks(df_report.index)
ax.set_xticklabels(df_report["week"].values)
cmap = plt.cm.coolwarm
custom_lines = [Line2D([0], [0], color="red", lw=10),
                Line2D([0], [0], color="grey", lw=10),
                Line2D([0], [0], color="purple", lw=10)]
ax.legend(custom_lines, ['No action taken', 'Acknlowledged', 'Closed'])
plt.title('Weekly cohorts of community-reported GH issues', fontsize=25)


In [ ]:
df_issues["week_dt"].unique()
def issue_is_open_during_week(week_dt, x, column_tag: str):
    week_end_dt = week_dt+datetime.timedelta(days = 7)
    created_dt = dateutil.parser.parse(x["created_dt"])
    close_dt = None
    if not pd.isnull(x["close_dt"]):
        close_dt = dateutil.parser.parse(x["close_dt"])
    
    return x[column_tag] and created_dt < week_end_dt and (pd.isnull(close_dt) or close_dt > week_end_dt)
    

# This is a not a Pandas way to do it, but gets the job done. Would be lovely to find out an elegant way to do this.
week_open_issue_counts = []    
for week_dt_ref in df_issues[df_issues["created_dt"] > min_report_created_datetime_str]["week_dt"].unique():
    devrel_count = df_issues.apply(lambda x: issue_is_open_during_week(week_dt_ref, x, "is_devrel"), axis=1)
    core_engineering_queue_count = df_issues.apply(lambda x: issue_is_open_during_week(week_dt_ref, x, "is_core_engineering_queue"), axis=1)
    help_wanted_count = df_issues.apply(lambda x: issue_is_open_during_week(week_dt_ref, x, "is_help_wanted"), axis=1)
    triage_count = df_issues.apply(lambda x: issue_is_open_during_week(week_dt_ref, x, "is_triage"), axis=1)
    
    week_open_issue_counts.append({
        "week_dt": week_dt_ref,
        "devrel_count": devrel_count[devrel_count == True].shape[0],
        "core_engineering_queue_count": core_engineering_queue_count[core_engineering_queue_count == True].shape[0],
        "help_wanted_count": help_wanted_count[help_wanted_count == True].shape[0],
        "triage_count": triage_count[triage_count == True].shape[0], 
    })

df_num_open_core_team_priority_issues_weekly = pd.DataFrame(week_open_issue_counts).sort_values("week_dt")
df_num_open_core_team_priority_issues_weekly["week"] = df_num_open_core_team_priority_issues_weekly["week_dt"].apply(lambda x: datetime.datetime.strftime(x, "%Y-%m-%d"))
df_num_open_core_team_priority_issues_weekly

In [ ]:
from matplotlib.lines import Line2D
plt.rcParams["figure.figsize"] = (20,7)
_, ax = plt.subplots()
plt.legend(bbox_to_anchor=(1.0, 1), loc='upper left')

for label in [
    "devrel_count",
    "core_engineering_queue_count",
    "help_wanted_count",
    "triage_count"
]:
    df_num_open_core_team_priority_issues_weekly.plot(kind='line', ax=ax, x="week", y=label, rot=90)

ax.set_xticks(df_report.index)
ax.set_xticklabels(df_num_open_core_team_priority_issues_weekly["week"].values)
cmap = plt.cm.coolwarm
plt.title('Number of open community issues', fontsize=25)

In [ ]:
# identify the week an issue was created, merged, and closed if not merged
issue_burn_date_df = pd.DataFrame(issue_info_rows)
issue_burn_date_df = issue_burn_date_df[["created_dt", "close_dt"]]
issue_burn_date_df["created_this_week"] = issue_burn_date_df.apply(lambda x: sunday_of_the_week_date_str(dateutil.parser.parse(x["created_dt"])), axis=1)
issue_burn_date_df["closed_this_week"] = issue_burn_date_df.apply(lambda x: sunday_of_the_week_date_str(dateutil.parser.parse(x["close_dt"]) if x["close_dt"] is not None else None), axis=1)
issue_burn_date_df = issue_burn_date_df[["created_this_week", "closed_this_week"]]

# count each category out
all_date_values = issue_burn_date_df.values.ravel()
unique_dates = pd.unique(all_date_values)
unique_dates = np.sort(unique_dates[unique_dates != np.array(None)])
issue_burn_df = pd.DataFrame(data=unique_dates, columns=["week"])
created_count = issue_burn_date_df["created_this_week"].value_counts()
closed_count = issue_burn_date_df["closed_this_week"].value_counts()
issue_burn_df = issue_burn_df.merge(created_count, how="outer", left_on="week", right_index=True)
issue_burn_df = issue_burn_df.merge(closed_count, how="outer", left_on="week", right_index=True)
issue_burn_df = issue_burn_df.fillna(0)

# calculate cumulative open issues
open_issues_start_of_first_week = 1
issue_burn_df["change_in_open_issues"] = issue_burn_df["created_this_week"] - issue_burn_df["closed_this_week"]
issue_burn_df["cumulative_open_issues"] = issue_burn_df["change_in_open_issues"].cumsum()
issue_burn_df["cumulative_open_issues"] = issue_burn_df["cumulative_open_issues"] + open_issues_start_of_first_week
issue_burn_df = issue_burn_df.iloc[-13:]
issue_burn_df

In [ ]:
# dataframe just for plot
issue_plot_df = issue_burn_df[["week", "created_this_week", "closed_this_week", "change_in_open_issues", "cumulative_open_issues"]]
issue_plot_df["closed_this_week"] = issue_plot_df["closed_this_week"] * -1
column_names = ["Week", "Created this Week", "Closed this Week", "Change in Open Issues", "Cumulative Open Issues"]
issue_plot_df.columns = column_names

# plot
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={"height_ratios": [4, 1]})

issue_plot_df.plot(x="Week", y="Cumulative Open Issues", kind="line", ax=ax1, drawstyle='steps-mid', color="black")
issue_plot_df.plot(x="Week", y="Created this Week", kind="bar", ax=ax1, color="salmon", bottom=issue_plot_df["Cumulative Open Issues"])
issue_plot_df.plot(x="Week", y="Closed this Week", kind="bar", ax=ax1, color="lightgreen", bottom=issue_plot_df["Cumulative Open Issues"])
issue_plot_df["min"] = issue_plot_df["Cumulative Open Issues"] + issue_plot_df["Closed this Week"]
issue_plot_df["max"] = issue_plot_df["Cumulative Open Issues"] + issue_plot_df["Created this Week"]
ax1.set_ylim([issue_plot_df["min"].min()-3, issue_plot_df["max"].max()+6])
issue_plot_df = issue_plot_df.drop(axis=1, columns=["max", "min"])
ax1.get_xaxis().set_visible(False)

change_colors = np.where(issue_plot_df["Change in Open Issues"] <= 0, "lightgreen", "salmon")
markers = np.where(issue_plot_df["Change in Open Issues"] <= 0, "v", "^")

issue_plot_df["dummy"] = [5] * len(issue_plot_df)
for i in range(len(issue_plot_df)):
    ax2.set_ylim([-5,10])
    ax2.scatter(x=issue_plot_df["Week"].iloc[i], y=issue_plot_df["dummy"].iloc[i], marker=markers[i], color=change_colors[i], s=500, edgecolors="black")
    ax2.annotate(text=issue_plot_df["Change in Open Issues"].iloc[i].astype(int), xy=(issue_plot_df["Week"].iloc[i], -2), ha="center", fontsize=20)
issue_plot_df = issue_plot_df.drop(axis=1, columns=["dummy"])
ax2.get_yaxis().set_visible(False)

fig.tight_layout()
ax1.grid(b=True, axis="y")
ax1.tick_params(axis="y", which="major", labelsize=16)
ax1.legend(loc=2, fontsize=16)
ax1.set_title('Cumulative Change in Open Community Issues (Previous 13 Weeks)', fontdict={'fontsize': 25})
ax2.tick_params(axis="x", which="major", labelsize=16, rotation=60)
plt.subplots_adjust(hspace=0)
plt.show()

In [ ]:
df_recent_issue_priority_categories = pd.DataFrame(df_issues[df_issues["week"] > min_report_created_datetime_str].groupby("priority_category")["number"].count())#.reset_index()
df_recent_issue_priority_categories.columns = ["count"]
df_recent_issue_priority_categories

In [ ]:
ax = df_recent_issue_priority_categories.plot(kind="pie",  y="count", autopct='%.1f')
plt.title('Issue Prioritization (reported by anyone)', fontsize=25)

ax.get_legend().remove()
